In [137]:
import wptools
import random
import pickle
import datetime
import matplotlib.pyplot as plt

In [87]:
cat = wptools.category('Category:2019_deaths')
cat.get_members()

en.wikipedia.org (categorymembers) Category:2019_deaths
en.wikipedia.org (categorymembers) Category:2019_deaths (&cmconti...
en.wikipedia.org (categorymembers) Category:2019_deaths (&cmconti...
en.wikipedia.org (categorymembers) Category:2019_deaths (&cmconti...
en.wikipedia.org (categorymembers) Category:2019_deaths (&cmconti...
en.wikipedia.org (categorymembers) Category:2019_deaths (&cmconti...
en.wikipedia.org (categorymembers) Category:2019_deaths (&cmconti...
en.wikipedia.org (categorymembers) Category:2019_deaths (&cmconti...
en.wikipedia.org (categorymembers) Category:2019_deaths (&cmconti...
en.wikipedia.org (categorymembers) Category:2019_deaths (&cmconti...
en.wikipedia.org (categorymembers) Category:2019_deaths (&cmconti...
en.wikipedia.org (categorymembers) Category:2019_deaths (&cmconti...
en.wikipedia.org (categorymembers) Category:2019_deaths (&cmconti...
en.wikipedia.org (categorymembers) Category:2019_deaths (&cmconti...
en.wikipedia.org (categorymembers) Category:201

In [88]:
print(len(cat.data['members']))
cat.data['members'][15:20]


8126


[{'pageid': 61779755, 'ns': 0, 'title': 'A.K.Subbaiah'},
 {'pageid': 39563088, 'ns': 0, 'title': 'Jane Aamund'},
 {'pageid': 62843632, 'ns': 0, 'title': 'Jüri Aarma'},
 {'pageid': 60238842, 'ns': 0, 'title': 'Sveinung Aarnseth'},
 {'pageid': 20317651, 'ns': 0, 'title': 'Chester Aaron'}]

In [94]:
%%capture
pages=[]
for i in range(20):
    page=wptools.page(pageid=cat.data['members'][random.randrange(15,8000)]['pageid'],timeout=0)
    page.get_parse()
    if(page.data.get('infobox') and 
       page.data['infobox'].get('birth_date') and 
       page.data['infobox'].get('death_date')):
        pages.append((page.data['infobox'].get('birth_date'), page.data['infobox'].get('death_date')))

with open("data.txt", "wb") as fp:
    pickle.dump(pages, fp)
    

In [95]:
with open("data.txt", "rb") as fp:  
    pages = pickle.load(fp)

In [96]:
def parser(birth,death):
    if(birth.lower().find("birth date")!=-1 and death.lower().find("death date")!=-1):
        birth=list(filter(lambda x:x!='',
                     ''.join(list(filter(lambda x:x.isdigit() or x=="|",birth))).split("|")))
        death=list(filter(lambda x:x!='',
                     ''.join(list(filter(lambda x:x.isdigit() or x=="|",death))).split("|")))[0:3]
        try:
            return (datetime.date(*map(int,birth)),datetime.date(*map(int,death)))
        except ValueError:
            return None
    return None

In [110]:
dates=list(filter(lambda x:x is not None,map(lambda page:parser(page[0], page[1]),pages)))  

print(len(pages),len(dates))

15 11


[(datetime.date(1925, 4, 21), datetime.date(2019, 7, 28)),
 (datetime.date(1938, 3, 5), datetime.date(2019, 7, 25)),
 (datetime.date(1955, 5, 10), datetime.date(2019, 4, 25)),
 (datetime.date(1938, 2, 5), datetime.date(2019, 2, 13)),
 (datetime.date(1934, 4, 5), datetime.date(2019, 3, 19)),
 (datetime.date(1925, 7, 18), datetime.date(2019, 1, 2)),
 (datetime.date(1915, 6, 26), datetime.date(2019, 4, 23)),
 (datetime.date(1969, 3, 31), datetime.date(2019, 9, 8)),
 (datetime.date(1941, 10, 9), datetime.date(2019, 9, 27)),
 (datetime.date(1955, 11, 7), datetime.date(2019, 3, 4))]

In [135]:
distanse_dates=list(map(lambda date:((date[0]-datetime.date(date[0].year,1,1))-
                                   (date[1]-datetime.date(date[1].year,1,1))).days,dates))
print(distanse_dates)

[-98, -142, 15, -8, 17, 197, 64, -161, 12, 248, 226]
